<div class="alert alert-block alert-success">
    <h1>
        Example notebook - 
    </h1>
    <p>
        
    </p>
</div>

## TODO

- clean up from workflow section
- documentation
- remove AI governance specific stuff from some functions in turingdb_kgsearch scripts

# Import modules and functions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import re
import pandas as pd
import networkx as nx
from sentence_transformers import SentenceTransformer
from IPython.display import display, Markdown

In [4]:
from turingdb.exceptions import TuringDBException

In [5]:
from turingdb_examples.utils import get_return_statements
from turingdb_examples.graph import build_create_command_from_networkx

In [6]:
from turingdb_kgsearch.embeddings import (
    build_node_only_embeddings,
    build_context_enriched_embeddings,
    build_smart_enriched_embeddings,
    build_sparse_embeddings,
    build_node2vec_embeddings,
)
from turingdb_kgsearch.search import (
    dense_search,
    sparse_search,
    print_results,
    hybrid_search,
    compare_search_methods,
)
from turingdb_kgsearch.subgraph import get_subgraph_around_query
from turingdb_kgsearch.visualization import (
    visualize_graph_with_pyvis,
    extract_and_visualize_subgraph,
)
from turingdb_kgsearch.workflow import (
    search_and_expand_hybrid_filtered,
    generate_report_hybrid_workflow_results,
)
from turingdb_kgsearch.statistics import get_subgraph_stats, print_subgraph_stats
from turingdb_kgsearch.ranking import (
    rank_nodes_by_importance,
    rank_nodes_by_importance_with_context,
    print_node_rankings,
    compare_node_importance,
    diagnose_rankings,
)
from turingdb_kgsearch.explain_results import (
    explain_retrieval,
    explain_top_results,
    print_explanation,
)
from turingdb_kgsearch.llm import (
    create_llm_prompt_with_graph,
    query_llm,
)

In [7]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

# Set path to data

In [8]:
example_name = "sec_8k_raw_text_filings_2024"
path_data = f"{os.getcwd()}/data/{example_name}"
if not os.path.exists(path_data):
    raise ValueError(f"{path_data} does not exists")

# Create graph using `turingdb` python package

<div class="alert alert-block alert-info">
    <h2>
        See <a href="https://docs.turingdb.ai/quickstart">TuringDB Get started documentation</a> for the important steps to follow :
    </h2>
    <h3>
        <ul>
            <li>Create your TuringDB account</li>
            <li>Create your instance in the <a href="https://console.turingdb.ai/auth">TuringDB Cloud UI</a></li>
            <li>Copy your Instance ID from the Database Instances management page</li>
            <li>Get API Key from the Settings in UI</li>
        </ul>
        Remember to have your instance active while working in this notebook !
    </h3>
</div>

## Connect to instance and transfer data

In [9]:
from turingdb import TuringDB

# Create TuringDB client
client = TuringDB(
    host="http://localhost:6666"  # Remove this parameter and set the two parameters below
    # instance_id=os.getenv("INSTANCE_ID"),
    # auth_token=os.getenv("AUTH_TOKEN"),
)

In [10]:
%%time

# Connect TuringDB client to s3 bucket
client.s3_connect(
    bucket_name="turing-internal",
    region="eu-west-2",
    access_key=os.getenv("AWS_ACCESS_KEY"),
    secret_key=os.getenv("AWS_SECRET_KEY"),
)

CPU times: user 260 ms, sys: 41.1 ms, total: 301 ms
Wall time: 342 ms


## Import and format data

In [11]:
# Build Graph from SEC 8-K Filings Dataset
"""
Build Graph from SEC 8-K Filings Dataset

Assumes CSV has columns like:
- company_name / cik (company identifier)
- filing_date / accession_number
- item_numbers (8-K items like 1.01, 2.02, etc.)
- text / filing_text (the actual filing content)
- ticker / symbol
- industry / sector
"""

import pandas as pd
import networkx as nx
from datetime import datetime

def build_sec_8k_graph(df, sample_size=None):
    """
    Build knowledge graph from SEC 8-K filings.
    
    Graph structure:
        Company nodes → Filed → Filing nodes → Contains → Item nodes
        Company nodes → Industry → Industry nodes
        Filing nodes → MentionsCompany → Other Company nodes (if mentioned)
    
    Args:
        csv_path: Path to CSV file
        sample_size: Number of rows to process (None = all)
    
    Returns:
        NetworkX DiGraph
    """
    
    # Inspect columns
    print(f"Columns: {list(df.columns)}")
    if sample_size is not None:
        print(f"Keep only {sample_size} first rows of dataframe")
        df = df.iloc[:sample_size, :].copy()
        print(df.shape)
        
    
    # Create graph
    G = nx.DiGraph()
    
    # Build graph based on available columns
    for idx, row in df.iterrows():
        if idx % 100 == 0:
            print(f"Processing filing {idx}/{len(df)}...")
        
        # Extract key fields (adapt to actual column names)
        company_id = str(row.get('cik', row.get('company_name', f'company_{idx}')))
        company_name = str(row.get('company_name', row.get('ticker', company_id)))
        filing_id = f"filing_{row.get('accession_number', idx)}"
        filing_date = row.get('filing_date', row.get('date', ''))
        ticker = row.get('ticker', row.get('symbol', ''))
        industry = row.get('industry', row.get('sector', ''))
        items = row.get('item_numbers', row.get('items', ''))
        text = row.get('text', row.get('filing_text', ''))
        
        # 1. Company node
        company_node = f"company_{company_id}"
        if company_node not in G:
            G.add_node(
                company_node,
                type='company',
                name=company_name,
                ticker=ticker if pd.notna(ticker) else '',
                industry=industry if pd.notna(industry) else ''
            )
        
        # 2. Filing node
        G.add_node(
            filing_id,
            type='filing',
            date=str(filing_date) if pd.notna(filing_date) else '',
            text=str(text)[:500] if pd.notna(text) else '',  # Truncate for memory
            company=company_name,
            ticker=ticker if pd.notna(ticker) else ''
        )
        
        # 3. Company filed Filing
        G.add_edge(company_node, filing_id, rel='filed')
        
        # 4. Industry node (if present)
        if pd.notna(industry) and industry:
            industry_node = f"industry_{industry}"
            if industry_node not in G:
                G.add_node(industry_node, type='industry', name=industry)
            G.add_edge(company_node, industry_node, rel='belongs_to')
        
        # 5. Item nodes (8-K items like 1.01, 2.02, etc.)
        if pd.notna(items) and items:
            # Items might be comma-separated or single
            item_list = str(items).split(',') if ',' in str(items) else [str(items)]
            
            for item in item_list:
                item = item.strip()
                if item:
                    item_node = f"item_{item}"
                    item_description = get_8k_item_description(item)
                    
                    if item_node not in G:
                        G.add_node(
                            item_node,
                            type='item',
                            item_number=item,
                            description=item_description
                        )
                    
                    G.add_edge(filing_id, item_node, rel='contains')
    
    print(f"\n✓ Graph built: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    
    # Print breakdown
    node_types = {}
    for node in G.nodes():
        ntype = G.nodes[node].get('type', 'unknown')
        node_types[ntype] = node_types.get(ntype, 0) + 1
    
    print("\nNode breakdown:")
    for ntype, count in node_types.items():
        print(f"  {ntype}: {count}")
    
    return G


def get_8k_item_description(item_number):
    """Map 8-K item numbers to descriptions."""
    
    items = {
        '1.01': 'Entry into Material Agreement',
        '1.02': 'Termination of Material Agreement',
        '1.03': 'Bankruptcy or Receivership',
        '1.04': 'Mine Safety Disclosure',
        '1.05': 'Material Cybersecurity Incidents',
        '2.01': 'Completion of Acquisition or Disposition',
        '2.02': 'Results of Operations and Financial Condition',
        '2.03': 'Creation of Direct Financial Obligation',
        '2.04': 'Triggering Events',
        '2.05': 'Costs Associated with Exit Activities',
        '2.06': 'Material Impairments',
        '3.01': 'Notice of Delisting',
        '3.02': 'Unregistered Sales of Equity Securities',
        '3.03': 'Material Modifications to Rights',
        '4.01': 'Changes in Control',
        '4.02': 'Non-Reliance on Previously Issued Statements',
        '5.01': 'Changes in Control of Registrant',
        '5.02': 'Departure/Election of Directors or Officers',
        '5.03': 'Amendments to Articles/Bylaws',
        '5.04': 'Temporary Suspension of Trading',
        '5.05': 'Amendment to Code of Ethics',
        '5.06': 'Change in Shell Company Status',
        '5.07': 'Submission of Matters to Vote',
        '5.08': 'Shareholder Nominations',
        '7.01': 'Regulation FD Disclosure',
        '8.01': 'Other Events',
        '9.01': 'Financial Statements and Exhibits'
    }
    
    return items.get(item_number, f'Item {item_number}')


def enhance_graph_with_temporal(G):
    """Add temporal connections between filings."""
    
    # Group filings by company
    filings_by_company = {}
    for node in G.nodes():
        if G.nodes[node].get('type') == 'filing':
            company = None
            # Find company that filed this
            for pred in G.predecessors(node):
                if G.nodes[pred].get('type') == 'company':
                    company = pred
                    break
            
            if company:
                if company not in filings_by_company:
                    filings_by_company[company] = []
                
                filing_date = G.nodes[node].get('date', '')
                filings_by_company[company].append((node, filing_date))
    
    # Sort and connect sequential filings
    for company, filings in filings_by_company.items():
        # Sort by date
        filings.sort(key=lambda x: x[1])
        
        # Connect sequential filings
        for i in range(len(filings) - 1):
            G.add_edge(filings[i][0], filings[i+1][0], rel='preceded_by')
    
    print(f"✓ Added temporal connections")
    return G


def find_company_mentions(G, text_column='text'):
    """
    Find companies mentioned in filing text.
    Creates MentionsCompany edges.
    """
    
    # Get all company names/tickers
    companies = {}
    for node in G.nodes():
        if G.nodes[node].get('type') == 'company':
            name = G.nodes[node].get('name', '')
            ticker = G.nodes[node].get('ticker', '')
            if name:
                companies[name.lower()] = node
            if ticker:
                companies[ticker.upper()] = node
    
    # Search for mentions
    mentions_added = 0
    for node in G.nodes():
        if G.nodes[node].get('type') == 'filing':
            text = G.nodes[node].get(text_column, '').lower()
            
            if not text:
                continue
            
            # Find mentioned companies
            for company_identifier, company_node in companies.items():
                if company_identifier in text:
                    # Don't link to self
                    filing_company = None
                    for pred in G.predecessors(node):
                        if G.nodes[pred].get('type') == 'company':
                            filing_company = pred
                            break
                    
                    if filing_company != company_node:
                        G.add_edge(node, company_node, rel='mentions')
                        mentions_added += 1
    
    print(f"✓ Added {mentions_added} company mention edges")
    return G

In [12]:
%%time

path_turing_folder = f"{os.getenv('HOME')}/.turing"
gml_filename = f"{example_name}.gml"

if os.path.exists(f"{path_data}/{gml_filename}"):
    print(f"""✓ gml file "{gml_filename}" already generated and saved locally\n""")
    G = nx.read_gml(f"{path_data}/{gml_filename}")
    print(f"{G}\n")
    nx.write_gml(G, f"{path_data}/{example_name}.gml")
else:
    print("Create graph from csv")
    # Load and format csv
    csv_filename = f"{example_name}.csv"
    df = pd.read_csv(f"{path_turing_folder}/data/{csv_filename}")
    df = df.replace({r"\s+$": "", r"^\s+": ""}, regex=True).replace(r"\n", " ", regex=True).replace({r"\\": ""}, regex=True)
    print(f"✓ Loaded {len(df)} rows from CSV")
    display(df)

    # Build graph
    G = build_sec_8k_graph(
        df,
        #sample_size=3000  # Start with some rows for testing
    )
    
    # Enhance with temporal connections
    G = enhance_graph_with_temporal(G)
    print(G)
    
    # Save graph
    nx.write_gml(G, f"{path_data}/{example_name}.gml") # f"{path_turing_folder}/data/{example_name}.gml"

✓ gml file "sec_8k_raw_text_filings_2024.gml" already generated and saved locally

DiGraph with 63723 nodes and 110655 edges

CPU times: user 7.96 s, sys: 199 ms, total: 8.16 s
Wall time: 8.1 s


## Transfer gml to instance

In [13]:
f"data/{example_name}/{gml_filename}"

'data/sec_8k_raw_text_filings_2024/sec_8k_raw_text_filings_2024.gml'

In [14]:
gml_filename

'sec_8k_raw_text_filings_2024.gml'

In [15]:
#%%time

gml_filename = f"{example_name}.gml"
client.transfer(
    src=f"data/{example_name}/{gml_filename}",
    dst=f"turingdb://{gml_filename}",  # to s3 bucket or TuringDB instance or local .turing
)

## Import and load `turingdb` graph

In [16]:
graph_name = example_name
print(f"Graph name : '{graph_name}'")

Graph name : 'sec_8k_raw_text_filings_2024'


In [17]:
%%time

try:
    client.query(f"""
    IMPORT GRAPH {graph_name}
    FROM "{graph_name}.gml"
    """)
except TuringDBException as e:
    print(f"""{e}""")

EXEC_ERROR: Failed to load graph sec_8k_raw_text_filings_2024
CPU times: user 900 μs, sys: 0 ns, total: 900 μs
Wall time: 845 μs


In [18]:
try:
    client.load_graph(example_name)
except TuringDBException as e:
    print(f"""{e}: Graph "{example_name}" already loaded""")

GRAPH_LOAD_ERROR: Graph "sec_8k_raw_text_filings_2024" already loaded


In [19]:
%%time

# Set graph
client.set_graph(graph_name)

CPU times: user 6 μs, sys: 0 ns, total: 6 μs
Wall time: 9.78 μs


<div class="alert alert-block alert-info">
    <h2>
        Visualize your graph in TuringDB Graph Visualizer ! Now that your instance is running:
    </h2>
    <h3>
        <ul>
            <li>Go to <a href="https://console.turingdb.ai/databases">TuringDB Console - Database Instances</a></li>
            <li>In your current instance panel, click on "Open Visualizer" button</li>
            <li>Visualizer opens, now you can choose your graph in the dropdown menu at the top-right corner</li>
        </ul>
        You can then play with your graph and visualize the nodes you want !
    </h3>
</div>

# Query TuringDB

## Use metaqueries to have insight on graph overall structure

<h3>
    To learn more about 📮 Metaqueries, please check TuringDB documentation on this <a href="https://turingdb.mintlify.app/query/cypher_subset#%F0%9F%93%AE-metaqueries">link</a>
</h3>

In [20]:
%%time

# CALL PROPERTIES() - returns a column of all the different node and edge properties and their types in the database
command = """
CALL PROPERTIES()
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = ["Property_ID", "Property_name", "Property_type"]
    display(df)

,Property_ID,Property_name,Property_type
0,0,label (String),String
1,1,type (String),String
2,2,name (String),String
3,3,ticker (String),String
4,4,industry (String),String
5,5,date (String),String
6,6,text (String),String
7,7,company (String),String
8,8,rel (String),String


CPU times: user 5.28 ms, sys: 23 μs, total: 5.3 ms
Wall time: 4.86 ms


In [21]:
%%time

# CALL LABELS () - returns a column of all the different node labels
command = """
CALL LABELS()
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = ["Node_type_ID", "Node_label"]
    display(df)

,Node_type_ID,Node_label
0,0,GMLNode


CPU times: user 3.46 ms, sys: 40 μs, total: 3.5 ms
Wall time: 2.99 ms


In [22]:
%%time

# CALL EDGETYPES() - returns a column of all the different edge types (edge equivalent of node labels)
command = """
CALL EDGETYPES()
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = ["Edge_type_ID", "Edge_label"]
    display(df)

,Edge_type_ID,Edge_label
0,0,GMLEdge


CPU times: user 3.26 ms, sys: 0 ns, total: 3.26 ms
Wall time: 2.79 ms


In [23]:
%%time

# CALL LABELSETS() - returns a two columns describing combinations of node labels
command = """
CALL LABELSETS()
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = ["Node_type_ID", "Node_label"]
    display(df)

,Node_type_ID,Node_label
0,0,GMLNode


CPU times: user 2.25 ms, sys: 1.02 ms, total: 3.28 ms
Wall time: 2.95 ms


## Simple queries

In [24]:
%%time

# Match all edges and return them
command = """
MATCH (n)-[e]-(m)
RETURN n, e, m
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,n,e,m
0,0,0,1284
1,1,1,1181
2,2,2,5420
3,3,3,118
4,4,4,1269
...,...,...,...
110650,63718,110650,63142
110651,63719,110651,63132
110652,63720,110652,53828
110653,63721,110653,50591


CPU times: user 22.8 ms, sys: 10.1 ms, total: 32.9 ms
Wall time: 35.2 ms


# Load the embedding model

In [25]:
%%time

# This will convert text to vectors
model = SentenceTransformer("paraphrase-MiniLM-L3-v2")
print(f"✓ Model loaded: {model.get_sentence_embedding_dimension()} dimensions")

✓ Model loaded: 384 dimensions
CPU times: user 122 ms, sys: 10.2 ms, total: 132 ms
Wall time: 1.37 s


# Build vector index on the graph

## Vector Search Implementation - Dense (semantic) search

#### How It Works

Each control is converted to a **384-dimensional vector** using a pre-trained language model (`paraphrase-MiniLM-L3-v2`).

**Search process:**
1. Convert user query to vector
2. Calculate cosine similarity with all control vectors
3. Rank by similarity score (0-1)
4. Return top-k most relevant controls

#### Why Vectors?

- **Semantic understanding**: "data protection" matches "privacy safeguards"
- **Handles synonyms**: "AI governance" finds "artificial intelligence oversight"
- **No keyword dependency**: Works even without exact term matches

In [26]:
from turingdb_kgsearch.save_embeddings import load_embeddings, save_embeddings

In [27]:
# Define embedding paths
embeddings_folder = "embeddings"

### Use different approaches

In [28]:
%%time

# Node-only embeddings
method = "node_only"
embeddings_file_path = f"{path_data}/{embeddings_folder}/embeddings_{method}.npz"

if os.path.exists(embeddings_file_path):
    print(f"Loading pre-computed embeddings (method {method})...")
    node_vectors_node_only, node_texts_node_only, metadata_node_only, _ = load_embeddings(embeddings_file_path)
else:
    print(f"Building (method {method}) embeddings (this may take a while)...")
    node_vectors_node_only, node_texts_node_only = build_node_only_embeddings(G, model)
    save_embeddings(
        node_vectors_node_only,
        node_texts_node_only,
        filepath=embeddings_file_path,
        embedding_type="dense"
    )

print("\n" + "=" * 80 + "\n")

Building (method node_only) embeddings (this may take a while)...
✓ Vector index built using node-only embeddings approach: 63723 vectors
✓ Embeddings saved to: /home/dev/turingdb-kgsearch/notebooks/data/sec_8k_raw_text_filings_2024/embeddings/embeddings_node_only.npz
  - Type: dense
  - Format: Dense
  - 63723 vectors
  - Vector dimension: 384
  - Has texts: Yes
  - Has vectorizer: No
  - File size: 87.24 MB


CPU times: user 4min 37s, sys: 8.83 s, total: 4min 46s
Wall time: 24.6 s


In [29]:
%%time

# Context-enriched embeddings
strategy = "heavy"  # "lightweight"
method = f"context_enriched_{strategy}"
embeddings_file_path = f"{path_data}/{embeddings_folder}/embeddings_{method}.npz"

if os.path.exists(embeddings_file_path):
    print(f"Loading pre-computed embeddings (method {method})...")
    node_vectors_context_heavy, node_texts_context_heavy, metadata, _ = load_embeddings(embeddings_file_path)
else:
    print(f"Building (method {method}) embeddings (this may take a while)...")
    node_vectors_context_heavy, node_texts_context_heavy = build_context_enriched_embeddings(
        G, model, strategy=strategy
    )
    save_embeddings(
        node_vectors_context_heavy,
        node_texts_context_heavy,
        filepath=embeddings_file_path,
        embedding_type="dense"
    )

print("\n" + "=" * 80 + "\n")

Building (method context_enriched_heavy) embeddings (this may take a while)...
✓ Vector index built using context-enriched embeddings approach (strategy heavy): 63723 vectors
✓ Embeddings saved to: /home/dev/turingdb-kgsearch/notebooks/data/sec_8k_raw_text_filings_2024/embeddings/embeddings_context_enriched_heavy.npz
  - Type: dense
  - Format: Dense
  - 63723 vectors
  - Vector dimension: 384
  - Has texts: Yes
  - Has vectorizer: No
  - File size: 87.24 MB


CPU times: user 4min 41s, sys: 8.85 s, total: 4min 50s
Wall time: 24.8 s


## Vector Search Implementation - Sparse (keyword) search

In [31]:
%%time

# Choose from which dense method take the created node_texts
node_texts = node_texts_context_heavy

# Build sparse embeddings
method = "sparse"
embeddings_file_path = f"{path_data}/{embeddings_folder}/embeddings_{method}.npz"

if os.path.exists(embeddings_file_path):
    print(f"Loading pre-computed embeddings (method {method})...")
    node_vectors_sparse, node_texts, metadata, vectorizer_sparse = load_embeddings(embeddings_file_path)
else:
    print(f"Building (method {method}) embeddings (this may take a while)...")
    node_vectors_sparse, _, vectorizer_sparse = build_sparse_embeddings(
        G=G, node_texts=node_texts  # max_features=500, ngram_range=(1, 2)
    )
    save_embeddings(
        node_vectors_sparse,
        node_texts,
        filepath=embeddings_file_path,
        embedding_type="sparse",
        vectorizer=vectorizer_sparse
    )

print("\n" + "=" * 80 + "\n")

Loading pre-computed embeddings (method sparse)...
is_sparse: True
sparse_filepath: /home/dev/turingdb-kgsearch/notebooks/data/sec_8k_raw_text_filings_2024/embeddings/embeddings_sparse_sparse.npz
✓ Embeddings loaded from: /home/dev/turingdb-kgsearch/notebooks/data/sec_8k_raw_text_filings_2024/embeddings/embeddings_sparse.npz
  - Type: sparse
  - Format: Sparse
  - 63723 vectors
  - Vector dimension: 500
  - Has texts: Yes
  - Has vectorizer: Yes


CPU times: user 2.51 s, sys: 438 ms, total: 2.95 s
Wall time: 2.47 s


## Node2Vec

In [33]:
%%time

# Build Node2Vec embeddings
method = "node2vec"
embeddings_file_path = f"{path_data}/{embeddings_folder}/embeddings_{method}.npz"

if os.path.exists(embeddings_file_path):
    print(f"Loading pre-computed embeddings (method {method})...")
    node_vectors_node2vec, _, metadata, _ = load_embeddings(embeddings_file_path)
else:
    print(f"Building (method {method}) embeddings (this may take a while)...")
    if node_vectors_node2vec is None:
        node_vectors_node2vec = build_node2vec_embeddings(G, dimensions=384)
    save_embeddings(
        node_vectors_node2vec,
        None,
        embeddings_file_path,
        "node2vec"
    )

print("\n" + "=" * 80 + "\n")

Loading pre-computed embeddings (method node2vec)...
✓ Embeddings loaded from: /home/dev/turingdb-kgsearch/notebooks/data/sec_8k_raw_text_filings_2024/embeddings/embeddings_node2vec.npz
  - Type: node2vec
  - Format: Dense
  - 63723 vectors
  - Vector dimension: 384
  - Has texts: No
  - Has vectorizer: No


CPU times: user 1.07 s, sys: 19 ms, total: 1.09 s
Wall time: 1.09 s


# Search capabilities

## Vector Search Implementation - Dense (semantic) search

Find controls relevant to any natural language query:

```python
results = search("data privacy protection", k=5)
```

**Use cases:**
- Exploratory research: "What controls cover AI model governance?"
- Concept-based lookup: "security monitoring requirements"
- Gap analysis: "What's missing in our risk management?"

### Query

In [34]:
%%time

# Try different queries
queries = [
    "Companies investing in AI",
    "Events related to AI regulation",
    "AI providers",
    "companies with risks of failure due to cloud providers",
    "AI model governance",
    "security monitoring requirements",
    "companies with risks linked to goods transportations/delivery",
]

for query in queries:
    print(f"\n{'=' * 80}")
    print(f"QUERY: '{query}'")
    print("=" * 80)
    results = dense_search(
        query=query,
        node_vectors=node_vectors_context_heavy,
        node_texts=node_texts,
        G=G,
        model=model,
        k=3
    )
    print_results(results)


QUERY: 'Companies investing in AI'

Found 3 results:

1. Similarity: 0.6553
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

2. Similarity: 0.6039
   Node: company_Investcorp AI Acquisition Corp.
   Type: company
   Text: company_Investcorp AI Acquisition Corp....

3. Similarity: 0.5871
   Node: company_AiAdvertising, Inc.
   Type: company
   Text: company_AiAdvertising, Inc....

QUERY: 'Events related to AI regulation'

Found 3 results:

1. Similarity: 0.4123
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

2. Similarity: 0.4049
   Node: company_Healthcare AI Acquisition Corp.
   Type: company
   Text: company_Healthcare AI Acquisition Corp....

3. Similarity: 0.3904
   Node: company_C3.ai, Inc.
   Type: company
   Text: company_C3.ai, Inc....

QUERY: 'AI providers'

Found 3 results:

1. Similarity: 0.7169
   Node: company_Healthcare AI Acquisition Corp.
   Type: company
   Text: company_Healthcare AI Acquisi

## Vector Search Implementation - Sparse (keyword) search

In [35]:
%%time

# Try different queries
for query in queries:
    print(f"\n{'=' * 80}")
    print(f"QUERY: '{query}'")
    print("=" * 80)
    results = sparse_search(
        query=query,
        node_vectors=node_vectors_sparse,
        node_texts=node_texts,  # same node texts used previously in dense search
        vectorizer=vectorizer_sparse,
        G=G,
        k=3
    )
    print_results(results)


QUERY: 'Companies investing in AI'

Found 3 results:

1. Similarity: 0.7194
   Node: company_C3.ai, Inc.
   Type: company
   Text: company_C3.ai, Inc....

2. Similarity: 0.7194
   Node: company_Jet.AI Inc.
   Type: company
   Text: company_Jet.AI Inc....

3. Similarity: 0.7194
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

QUERY: 'Events related to AI regulation'

Found 3 results:

1. Similarity: 1.0000
   Node: company_C3.ai, Inc.
   Type: company
   Text: company_C3.ai, Inc....

2. Similarity: 1.0000
   Node: company_Jet.AI Inc.
   Type: company
   Text: company_Jet.AI Inc....

3. Similarity: 1.0000
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

QUERY: 'AI providers'

Found 3 results:

1. Similarity: 1.0000
   Node: company_C3.ai, Inc.
   Type: company
   Text: company_C3.ai, Inc....

2. Similarity: 1.0000
   Node: company_Jet.AI Inc.
   Type: company
   Text: company_Jet.AI Inc....

3. Similarity: 1.000

## Hybrid search: Best of Both Worlds

### The Problem

- **Dense (semantic) search**: Great for concepts, misses exact terms
- **Sparse (keyword) search**: Finds exact matches, misses semantics

### The Solution

**Hybrid search** combines both approaches:

```
final_score = α × semantic_score + (1-α) × keyword_score
```

### Alpha Parameter Guide

| Alpha | Behavior | Best For |
|-------|----------|----------|
| 1.0 | Pure semantic | Conceptual queries |
| 0.7 | Favor semantics | General use (recommended) |
| 0.5 | Balanced | Mixed queries |
| 0.3 | Favor keywords | Technical lookups |
| 0.0 | Pure keywords | Exact term matching |

### When to Use What

**Dense (α=1.0)**
- Query: "What covers security?"
- Finds: Controls about protection, safeguards, defense

**Sparse (α=0.0)**
- Query: "ISO27001 A.8.24"
- Finds: Exact standard reference

**Hybrid (α=0.7)**
- Query: "NIST risk management frameworks"
- Finds: Both NIST references AND risk-related controls

### Query

In [36]:
%%time

# Try different queries
for query in queries:
    print(f"\n{'=' * 80}")
    print(f"QUERY: '{query}'")
    print("=" * 80)
    results = hybrid_search(
        query=query,
        G=G,
        dense_node_vectors=node_vectors_context_heavy,
        sparse_node_vectors=node_vectors_sparse,
        sparse_vectorizer=vectorizer_sparse,
        node_texts=node_texts,  # same node texts used for both dense and sparse
        model=model,
        k=3,
        alpha=0.7,  # 70% semantic, 30% keywords
    )
    print_results(results)


QUERY: 'Companies investing in AI'

Found 3 results:

1. Similarity: 1.0000
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

2. Similarity: 0.4919
   Node: company_Odysight.ai Inc.
   Type: company
   Text: company_Odysight.ai Inc....

3. Similarity: 0.3828
   Node: company_Investcorp AI Acquisition Corp.
   Type: company
   Text: company_Investcorp AI Acquisition Corp....

QUERY: 'Events related to AI regulation'

Found 3 results:

1. Similarity: 1.0000
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

2. Similarity: 0.6496
   Node: company_C3.ai, Inc.
   Type: company
   Text: company_C3.ai, Inc....

3. Similarity: 0.6117
   Node: company_NEURALBASE AI LTD.
   Type: company
   Text: company_NEURALBASE AI LTD....

QUERY: 'AI providers'

Found 3 results:

1. Similarity: 0.7000
   Node: company_Healthcare AI Acquisition Corp.
   Type: company
   Text: company_Healthcare AI Acquisition Corp....

2. Similarity: 0.

## Compare Dense vs Sparse vs Hybrid

In [37]:
%%time

# Limit results to specific node type (all node types by default)
k = 5
alpha = 0.7
node_type = None

for query in queries:
    compare_search_methods(
        query=query,
        G=G,
        dense_node_vectors=node_vectors_context_heavy,
        sparse_node_vectors=node_vectors_sparse,
        sparse_vectorizer=vectorizer_sparse,
        node_texts=node_texts,
        model=model,
        k=k,
        alpha=alpha,
        node_type=node_type
    )


QUERY: 'Companies investing in AI'

1. DENSE ONLY (Semantic):
--------------------------------------------------------------------------------
1. 0.655 | company_Global AI, Inc....
2. 0.604 | company_Investcorp AI Acquisition Corp....
3. 0.587 | company_AiAdvertising, Inc....

2. SPARSE ONLY (Keywords):
--------------------------------------------------------------------------------
1. 0.719 | company_C3.ai, Inc....
2. 0.719 | company_Jet.AI Inc....
3. 0.719 | company_Global AI, Inc....

3. HYBRID (alpha=0.7):
--------------------------------------------------------------------------------
1. 1.000 (raw: D:0.66/S:0.72) | (norm: D:1.00/S:1.00) | company_Global AI, Inc....
2. 0.539 (raw: D:0.57/S:0.72) | (norm: D:0.34/S:1.00) | company_Odysight.ai Inc....
3. 0.412 (raw: D:0.60/S:0.00) | (norm: D:0.59/S:0.00) | company_Investcorp AI Acquisition Corp....

QUERY: 'Events related to AI regulation'

1. DENSE ONLY (Semantic):
-------------------------------------------------------------------

# Get subgraph around query results and visualise

### Graph-Based Context Retrieval

Get full context around relevant controls:

```python
subgraph = get_subgraph_around_query("risk management", k=3, hops=1)
```

**Returns:**
- Relevant controls
- Related topics and domains
- All mapped standard references
- Network connections

In [38]:
query = queries[1]

In [39]:
print(f"Query: '{query}'")

method_to_test = "hybrid"
possible_method_to_test = ["dense", "sparse", "hybrid"]
if method_to_test not in possible_method_to_test:
    raise ValueError(f"method_to_test has to be one of {possible_method_to_test}")

if method_to_test == "dense":
    # Dense search
    subgraph, results = get_subgraph_around_query(
        query=query,
        G=G,
        search_func=dense_search,
        search_params={
            "node_vectors": node_vectors_context_heavy,
            "node_texts": node_texts,
            "model": model
        },
        k=3,
        hops=1,
    )

elif method_to_test == "sparse":
    # Sparse search
    subgraph, results = get_subgraph_around_query(
        query=query,
        G=G,
        search_func=sparse_search,
        search_params={
            "sparse_vectors": node_vectors_sparse,
            "sparse_vectorizer": vectorizer_sparse,
            "node_texts": node_texts
        },
        k=3,
        hops=1,
    )

elif method_to_test == "hybrid":
    # Hybrid search
    subgraph, results = get_subgraph_around_query(
        query=query,
        G=G,
        search_func=hybrid_search,
        search_params={
            "dense_node_vectors": node_vectors_context_heavy,
            "sparse_node_vectors": node_vectors_sparse,
            "sparse_vectorizer": vectorizer_sparse,
            "node_texts": node_texts,  # same node texts used for both dense and sparse
            "model": model,
            "alpha": 0.7
        },
        k=3,
        hops=1,
    )

Query: 'Events related to AI regulation'


In [40]:
print(f"Query: '{query}'")
print(
    f"Subgraph: {subgraph.number_of_nodes()} nodes, {subgraph.number_of_edges()} edges"
)

# Show what's in the subgraph
types_in_subgraph = {}
for node, data in subgraph.nodes(data=True):
    ntype = data.get("type", "unknown")
    types_in_subgraph[ntype] = types_in_subgraph.get(ntype, 0) + 1

print("\nSubgraph composition:")
for ntype, count in types_in_subgraph.items():
    print(f"  {ntype}: {count}")

Query: 'Events related to AI regulation'
Subgraph: 17 nodes, 25 edges

Subgraph composition:
  filing: 14
  company: 3


# Visualization Capabilities

## Interactive Visualization (PyVis)

- Hover to see full control text
- Click to explore connections
- Physics-based layout
- Filterable and zoomable

**Features:**
- Color legend for node types
- Relevance-based sizing
- Relationship labels on edges
- Responsive browser-based interface

In [42]:
print(f"Query: '{query}'")

# With hybrid search
extract_and_visualize_subgraph(
    query=query,
    G=G,
    search_func=hybrid_search,
    search_params={
        "dense_node_vectors": node_vectors_context_heavy,
        "sparse_node_vectors": node_vectors_sparse,
        "sparse_vectorizer": vectorizer_sparse,
        "node_texts": node_texts,  # same node texts used for both dense and sparse
        "model": model,
        "alpha": 0.7
    },
    k=4,
    hops=2,
    output_file=f"{example_name}.html"  # f"{path_data}/{example_name}.html",
)

Query: 'Events related to AI regulation'
Auto-generated color map for 2 node types:
  company: #ff6b6b
  filing: #4ecdc4
✓ Interactive graph saved to: sec_8k_raw_text_filings_2024.html
  Nodes: 28
  Edges: 44


# Workflow

In [48]:
%%time

print(f"Query: '{query}'")

semantic_results, expanded, subgraph = search_and_expand_hybrid_filtered(
    query=query,
    G=G,
    node_vectors=node_vectors_context_heavy,
    node_texts=node_texts,
    sparse_vectors=node_vectors_sparse,
    sparse_vectorizer=vectorizer_sparse,
    structural_vectors=node_vectors_node2vec,
    model=model,
    k_search=5,
    max_hops=10,
    min_structural_sim=0.1,  # Must be structurally similar
    min_semantic_sim=0.1,  # AND semantically relevant to query
    structural_weight=0.5,  # 50-50 balance
    alpha=0.7,  # Weight alpha to attribute to semantic (dense) search, (1 - alpha) for keyword (sparse) search
)

report = generate_report_hybrid_workflow_results(semantic_results, expanded)
print(report)

Query: 'Events related to AI regulation'
Stage 1: Hybrid search for 'Events related to AI regulation'...
--------------------------------------------------------------------------------

Found 5 semantically relevant seed nodes:
  1. company_Global AI, Inc. (score: 1.000)
     company_Global AI, Inc....
  2. company_C3.ai, Inc. (score: 0.853)
     company_C3.ai, Inc....
  3. company_NEURALBASE AI LTD. (score: 0.837)
     company_NEURALBASE AI LTD....
  4. company_Odysight.ai Inc. (score: 0.728)
     company_Odysight.ai Inc....
  5. company_Jet.AI Inc. (score: 0.706)
     company_Jet.AI Inc....

Stage 2: Hybrid filtering (structural + semantic)...
  - Max hops: 10
  - Min structural similarity: 0.1
  - Min semantic similarity: 0.1
  - Structural weight: 0.5
--------------------------------------------------------------------------------

  Expanding from: company_Global AI, Inc.
  Found 0 neighbors (after hybrid filtering):

  Expanding from: company_C3.ai, Inc.
  Found 0 neighbors (aft

In [75]:
# Access subgraph data
print(f"\nSubgraph ({subgraph}) node attributes:")
for node in list(subgraph.nodes()):
    print(f"\n* {node}:")
    for key, value in subgraph.nodes[node].items():
        if key not in ["statement"]:  # Skip long text
            print(f"  {key}: {value}")

# Export subgraph if needed
# nx.write_gml(subgraph, "filtered_subgraph.gml")

# Visualise subgraph
visualize_graph_with_pyvis(subgraph, output_file=f"{example_name}_expanded.html")


Subgraph (DiGraph with 14 nodes and 17 edges) node attributes:

* filing_19063:
  type: filing
  date: 
  text: 
  company: Odysight.ai Inc.
  ticker: ODYS
  is_seed: False
  is_intermediate: True

* company_NEURALBASE AI LTD.:
  type: company
  name: NEURALBASE AI LTD.
  ticker: VIRA
  industry: 
  is_seed: True
  seed_score: 0.8370429659711837
  dense_score: 0.38801097869873047
  sparse_score: 1.0

* filing_20699:
  type: filing
  date: 
  text: 
  company: Odysight.ai Inc.
  ticker: ODYS
  is_seed: False
  is_intermediate: True

* filing_23742:
  type: filing
  date: 
  text: 
  company: Odysight.ai Inc.
  ticker: ODYS
  is_seed: False
  is_intermediate: True

* company_Odysight.ai Inc.:
  type: company
  name: Odysight.ai Inc.
  ticker: ODYS
  industry: 
  is_seed: True
  seed_score: 0.7275194744085485
  dense_score: 0.37166985869407654
  sparse_score: 1.0

* company_Global AI, Inc.:
  type: company
  name: Global AI, Inc.
  ticker: GLAI
  industry: 
  is_seed: True
  seed_score: 

In [55]:
for node in list(G.nodes)[:50]:
    print(node)

company_Tesla, Inc.
filing_0
company_BERKSHIRE HATHAWAY INC
filing_1
company_STEEL PARTNERS HOLDINGS L.P.
filing_2
company_Gryphon Digital Mining, Inc.
filing_3
company_FOCUS UNIVERSAL INC.
filing_4
company_KINGSTONE COMPANIES, INC.
filing_5
company_NEWS CORP
filing_6
company_GoDaddy Inc.
filing_7
company_Hilton Grand Vacations Inc.
filing_8
company_EchoStar CORP
filing_9
company_Quanterix Corp
filing_10
company_BlackRock TCP Capital Corp.
filing_11
company_CENTERPOINT ENERGY INC
filing_12
company_Western Acquisition Ventures Corp.
filing_13
company_Praxis Precision Medicines, Inc.
filing_14
company_Infinera Corp
filing_15
company_BARNES GROUP INC
filing_16
company_EXPAND ENERGY Corp
filing_17
company_MeridianLink, Inc.
filing_18
company_WOLVERINE WORLD WIDE INC /DE/
filing_19
company_Transocean Ltd.
filing_20
company_Osprey Bitcoin Trust
filing_21
company_Polaris Inc.
filing_22
company_C4 Therapeutics, Inc.
filing_23
company_ASPEN AEROGELS INC
filing_24


# Results exploration

## Graph statistics

In [50]:
# Usage
stats = get_subgraph_stats(
    subgraph, include_node_breakdown=True, include_centrality=True, include_paths=True
)
print_subgraph_stats(stats, verbose=True)


SUBGRAPH STATISTICS

📊 Basic Metrics:
   nodes: 14
   edges: 17
   density: 0.09340659340659341
   is_connected: False

🔗 Degree Statistics:
   average: 2.43
   max: 9.00
   min: 0.00
   median: 3.00

🏷️  Node Types:
   filing: 9
   company: 5

🎯 Node Roles:
   seed: 5
   found: 2
   intermediate: 7

⭐ Most Central Nodes:
   By Degree:
      company_Odysight.ai Inc.: 0.692
      filing_19063: 0.231
      filing_20699: 0.231
      filing_23742: 0.231
      filing_22599: 0.231
   By Betweenness:
      filing_22599: 0.103
      filing_20699: 0.096
      filing_23742: 0.096
      filing_19063: 0.077
      filing_31180: 0.077

🔗 Edge Types:
   preceded_by: 8
   filed: 9

📈 Relevance Scores:
   avg_score: 0.571
   max_score: 0.571
   min_score: 0.571
   nodes_with_scores: 2



## Node importance ranking

In [59]:
node_to_check = "company_Odysight.ai Inc."
focus_type = "company"

In [60]:
# Usage examples
rankings = rank_nodes_by_importance(
    subgraph,
    methods="all",  # or ['pagerank', 'degree', 'relevance']
    top_k=10,
    aggregate="average",  # or 'max' or {'pagerank': 0.4, 'degree': 0.3, 'relevance': 0.3}
)

print_node_rankings(rankings, subgraph, show_details=True)


NODE IMPORTANCE RANKINGS

Ranked 14 nodes using: pagerank, degree, betweenness, eigenvector, relevance


📊 PAGERANK (Top 5):
   1. filing_38371 (filing): 0.1441
   2. filing_31454 (filing): 0.1365
   3. filing_31180 (filing): 0.1274
   4. filing_23742 (filing): 0.1168
   5. filing_22599 (filing): 0.1043

📊 DEGREE (Top 5):
   1. company_Odysight.ai Inc. (company): 0.6923
   2. filing_19063 (filing): 0.2308
   3. filing_20699 (filing): 0.2308
   4. filing_23742 (filing): 0.2308
   5. filing_22599 (filing): 0.2308

📊 BETWEENNESS (Top 5):
   1. filing_22599 (filing): 0.1026
   2. filing_20699 (filing): 0.0962
   3. filing_23742 (filing): 0.0962
   4. filing_19063 (filing): 0.0769
   5. filing_31180 (filing): 0.0769

❌ CLOSENESS: graph is not connected enough to compute closeness centrality

📊 EIGENVECTOR (Top 5):
   1. filing_38371 (filing): 0.9999
   2. filing_31454 (filing): 0.0110
   3. filing_31180 (filing): 0.0001
   4. filing_23742 (filing): 0.0000
   5. filing_22599 (filing): 0.000

In [61]:
# Compare specific node
# Check if node exists first
if node_to_check in subgraph:
    node_comparison = compare_node_importance(node_to_check, rankings)
    if node_comparison:
        print(f"\nHow {node_to_check} ranks:")
        for method, info in node_comparison.items():
            print(f"  {method}: #{info['rank']} (score: {info['score']:.3f})")
    else:
        print(f"\{node_to_check} not in top rankings")
else:
    print(f"\{node_to_check} not in subgraph")


How company_Odysight.ai Inc. ranks:
  degree: #1 (score: 0.692)
  betweenness: #9 (score: 0.000)


In [62]:
filtered_rankings, full_rankings = rank_nodes_by_importance_with_context(
    subgraph,
    focus_type=focus_type,
    methods="all",
    top_k=10,
    aggregate="average",  # or 'max' or {'pagerank': 0.4, 'degree': 0.3, 'relevance': 0.3}
)

print("\n=== CONTROL NODE RANKINGS (computed on full graph) ===")
print_node_rankings(filtered_rankings, subgraph)


=== CONTROL NODE RANKINGS (computed on full graph) ===

NODE IMPORTANCE RANKINGS

Ranked 14 nodes using: pagerank, degree, betweenness, eigenvector, relevance


📊 PAGERANK (Top 5):
   1. company_NEURALBASE AI LTD. (company): 0.0257

📊 DEGREE (Top 5):
   1. company_Odysight.ai Inc. (company): 0.6923

📊 BETWEENNESS (Top 5):
   1. company_NEURALBASE AI LTD. (company): 0.0000
   2. company_Odysight.ai Inc. (company): 0.0000
   3. company_Global AI, Inc. (company): 0.0000

❌ CLOSENESS: graph is not connected enough to compute closeness centrality

📊 EIGENVECTOR (Top 5):
   1. company_NEURALBASE AI LTD. (company): 0.0000

📊 RELEVANCE (Top 5):

⭐ COMBINED RANKING (Top 10):
   1. company_Odysight.ai Inc. (company): 0.2500



In [63]:
# Debug why rankings might be zero for some nodes in some metrics
diagnose_rankings(subgraph, focus_type)

Total nodes: 14
Filtered nodes (company): 5
Filtered subgraph connected: False
Filtered subgraph edges: 0
Isolated nodes: 5/5

⚠️  WARNING: All filtered nodes are isolated!
   Recommendation: Don't filter by node_type for ranking.
   Rank on full graph, then filter results for display.


## Explain node retrieval

In [66]:
# For a specific node
explanation = explain_retrieval(
    node_id=node_to_check,
    query=query,
    subgraph=subgraph,
    node_vectors=node_vectors_context_heavy,
    sparse_vectors=node_vectors_sparse,
    sparse_vectorizer=vectorizer_sparse,
    structural_vectors=node_vectors_node2vec,
    model=model,
)
print_explanation(explanation, verbose=True)


RETRIEVAL EXPLANATION: company_Odysight.ai Inc.

Node Type: company
Query: 'Events related to AI regulation'

🎯 Reason: SEED NODE
   Initial search score: 0.728
   - Semantic component: 0.372
   - Keyword component: 1.000

📝 Semantic Similarity: 0.372
   Low similarity

🔍 Keyword Matching: 1.000
   Very high similarity
   Top TF-IDF terms in node:
      - ai: 1.000

🕸️  Structural Similarity:
   Shares graph structure with seed nodes

📄 Content Preview:
   Odysight.ai Inc.



In [68]:
explanations = explain_top_results(
    query=query,
    results=semantic_results,
    subgraph=subgraph,
    node_vectors=node_vectors_context_heavy,
    sparse_vectors=node_vectors_sparse,
    sparse_vectorizer=vectorizer_sparse,
    structural_vectors=node_vectors_node2vec,
    model=model,
    top_k=3,
)


RETRIEVAL EXPLANATION: company_Global AI, Inc.

Node Type: company
Query: 'Events related to AI regulation'

🎯 Reason: SEED NODE
   Initial search score: 1.000
   - Semantic component: 0.412
   - Keyword component: 1.000

📝 Semantic Similarity: 0.412
   Low similarity

🔍 Keyword Matching: 1.000
   Very high similarity
   Top TF-IDF terms in node:
      - ai: 1.000

🕸️  Structural Similarity:
   Shares graph structure with seed nodes

📄 Content Preview:
   Global AI, Inc.


RETRIEVAL EXPLANATION: company_C3.ai, Inc.

Node Type: company
Query: 'Events related to AI regulation'

🎯 Reason: SEED NODE
   Initial search score: 0.853
   - Semantic component: 0.390
   - Keyword component: 1.000

📝 Semantic Similarity: 0.390
   Low similarity

🔍 Keyword Matching: 1.000
   Very high similarity
   Top TF-IDF terms in node:
      - ai: 1.000

🕸️  Structural Similarity:
   Shares graph structure with seed nodes

📄 Content Preview:
   C3.ai, Inc.


RETRIEVAL EXPLANATION: company_NEURALBASE AI LTD.



# Take exploration results and ask LLM to answer original client query only using subgraph

In [69]:
api_keys = {
    "Anthropic": os.getenv("ANTHROPIC_API_KEY"),
    "Mistral": os.getenv("MISTRAL_API_KEY"),
    "OpenAI": os.getenv("OPENAI_API_KEY"),
}

## Create prompt

In [71]:
print(f"Query: '{query}'")

# Complete prompt
prompt = create_llm_prompt_with_graph(
    query=query,
    subgraph=subgraph,
    report=report,
    format="natural",  # or 'markdown'
)

# Send to LLM
print(prompt)

Query: 'Events related to AI regulation'
# Graph-Based Query Response

## User Query
"Events related to AI regulation"

## Search Results
HYBRID-FILTERED WORKFLOW RESULTS

1. SEED NODE (Hybrid Search Match):
   Node: company_Global AI, Inc.
   Semantic Score: 1.000
   Text: company_Global AI, Inc....

   HYBRID-FILTERED NEIGHBORS:
   (Must pass BOTH structural AND semantic thresholds)

2. SEED NODE (Hybrid Search Match):
   Node: company_C3.ai, Inc.
   Semantic Score: 0.853
   Text: company_C3.ai, Inc....

   HYBRID-FILTERED NEIGHBORS:
   (Must pass BOTH structural AND semantic thresholds)

3. SEED NODE (Hybrid Search Match):
   Node: company_NEURALBASE AI LTD.
   Semantic Score: 0.837
   Text: company_NEURALBASE AI LTD....

   HYBRID-FILTERED NEIGHBORS:
   (Must pass BOTH structural AND semantic thresholds)

4. SEED NODE (Hybrid Search Match):
   Node: company_Odysight.ai Inc.
   Semantic Score: 0.728
   Text: company_Odysight.ai Inc....

   HYBRID-FILTERED NEIGHBORS:
   (Must pass BO

## Query LLM

In [72]:
%%time

provider = "Anthropic"

result = query_llm(
    prompt=prompt,
    # system_prompt=system_prompt,
    provider=provider,
    api_key=api_keys[provider],
    temperature=0.2,
)

CPU times: user 402 ms, sys: 66.4 ms, total: 468 ms
Wall time: 8.48 s


In [73]:
display(Markdown(result))

Based on the provided graph data, here's an analysis of events related to AI regulation:

Key Observations:
1. AI Companies Identified:
- Global AI, Inc.
- C3.ai, Inc.
- NEURALBASE AI LTD.
- Odysight.ai Inc.
- Jet.AI Inc.

2. Filing Patterns:
- Odysight.ai Inc. has the most extensive filing history, with 12 filed documents
- The graph shows a complex network of interconnected filings with precedence relationships

3. Regulatory Context:
While the graph doesn't explicitly label these as "regulation" documents, the numerous filings suggest potential regulatory interactions or compliance activities among AI companies.

Insights:
- The graph reveals a dense network of AI companies with multiple filing connections
- Odysight.ai Inc. appears to be particularly active in documentation/filing
- There are sequential relationships between filings (e.g., filing_19063 preceded by filing_20699)

Limitations:
- The graph lacks explicit details about the nature of these filings
- No direct mention of specific regulatory events or policies

Conclusion:
The graph suggests ongoing administrative or regulatory activities within the AI industry, with multiple companies generating interconnected filing documentation. However, without more context, the specific nature of these events cannot be definitively determined.

Recommendation:
Further investigation would be needed to understand the precise regulatory implications of these filings.

In [74]:
print("Notebook finished !")

Notebook finished !
